### 이동경로 로그를 기반으로 가상 경로를 예측하는 RNN

In [1]:
import torch
import torch.nn as nn

In [2]:
# RNN model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size) # (5+128, 128)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input_tensor, hidden_tensor):
        combined = torch.cat((input_tensor, hidden_tensor), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

In [3]:
training_data = [("page1", "page2", "page3", "page4"),
                 ("page1", "page2", "page4"),
                 ("page1", "page2", "page3"),
                 ("page1", "page2", "page4", "page5")]

pages = ["page1", "page2", "page3", "page4", "page5"]
page_to_index = {pages[i] : i for i in range(len(pages))}
# {'page1': 0, 'page2': 1, 'page3': 2, 'page4': 3, 'page5': 4}

In [4]:
def sequence_to_tensor(sequence):
    tensor = torch.zeros(len(sequence), 1, len(pages)) # 4(input page 수) x 5(총 page 수) 0으로 채워짐
    '''
    ("page1", "page2", "page3", "page4") 
    =>  tensor([[[0., 0., 0., 0., 0.]], : page1
            [[0., 0., 0., 0., 0.]], : page2
            [[0., 0., 0., 0., 0.]], : page3
            [[0., 0., 0., 0., 0.]]]) : page4
            
    ("page1", "page2", "page3")
    => tensor([[[0., 0., 0., 0., 0.]], : page1
        [[0., 0., 0., 0., 0.]], : page2
        [[0., 0., 0., 0., 0.]]]) : page3
    '''
    
    for i, page in enumerate(sequence):
        tensor[i][0][page_to_index[page]] = 1 # i번째 텐서의 0번째 줄의 "page n"의 인덱스에 1
        '''
        ("page1", "page2", "page3")
        => tensor([[[1., 0., 0., 0., 0.]], : page1
        [[0., 1., 0., 0., 0.]], : page2
        [[0., 0., 1., 0., 0.]]]) : page3
        '''
    return tensor

In [59]:
training_tensors = [sequence_to_tensor(sequence) for sequence in training_data] # () 한 줄 한 줄 씩

In [63]:
i = 1
for tensor in training_tensors:
    print("tensor{} : {}".format(i, tensor))
    i += 1

tensor1 : tensor([[[1., 0., 0., 0., 0.]],

        [[0., 1., 0., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 0., 1., 0.]]])
tensor2 : tensor([[[1., 0., 0., 0., 0.]],

        [[0., 1., 0., 0., 0.]],

        [[0., 0., 0., 1., 0.]]])
tensor3 : tensor([[[1., 0., 0., 0., 0.]],

        [[0., 1., 0., 0., 0.]],

        [[0., 0., 1., 0., 0.]]])
tensor4 : tensor([[[1., 0., 0., 0., 0.]],

        [[0., 1., 0., 0., 0.]],

        [[0., 0., 0., 1., 0.]],

        [[0., 0., 0., 0., 1.]]])


In [83]:
training_tensors = [sequence_to_tensor(sequence) for sequence in training_data] # () 한 줄 한 줄 씩

input_size = len(pages) # 5
hidden_size = 128
output_size = len(pages)

rnn = RNN(input_size, hidden_size, output_size)

criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr = 0.001)

num_epochs = 1000
for epoch in range(num_epochs):
    for tensor in training_tensors:
        hidden = rnn.init_hidden()
        rnn.zero_grad()
        loss = 0
        
        # torch.Size([4, 1, 5])
        for i in range(tensor.size()[0]-1): # 첫번째 문장의 경우 4-1 = 3. 총 0, 1, 2 반복
            '''
            print(f"tensor {i} : {tensor[i]}") [[1., 0., 0., 0., 0.]]
            print(f'tensor {i} shape : {tensor[i].shape}') [1, 5]
            '''
            
            output, hidden = rnn(tensor[i], hidden)
            '''
            print(f'output : {output}') [[0., 1., 0., 0., 0.]]
            print(f'output.shape : {output.shape}') [1, 5]
            print(f'hidden : {hidden}')
            print(f'hidden.shape : {hidden.shape}') [1, 128]
            
            print(f"tensor {i+1} : {tensor[i+1]}") [[0., 1., 0., 0., 0.]]
            print(f'tensor {i+1} shape : {tensor[i+1].shape}') [1, 5]

            print(f'torch.argmax(tensor {i+1}): {torch.argmax(tensor[i+1])}')  tensor([1])
            print(f'torch.argmax(tensor {i+1}).shape: {tensor[i+1].shape}')  torch.Size([1])
            '''
            loss += criterion(output, torch.argmax(tensor[i+1], dim=1)) # tensor([1]). tensor[i+1] 중 가장 큰 인덱스 반환
            
        loss.backward()
        optimizer.step()
        
    if epoch % 100 == 0:
        print("Epoch: {}/{} | Loss: {:.4f}".format(epoch+1, num_epochs, loss.item()))
        
        
def predict(sequence):
    tensor = sequence_to_tensor(sequence)
    hidden = rnn.init_hidden()
    for i in range(tensor.size()[0]-1):
        output, hidden = rnn(tensor[i], hidden)
    _, topi = output.topk(1) # 텐서의 가장 큰 값 및 주소
    return pages[topi.item()] # 텐서에서 정수 값으로 변경

test_sequence = ("page1", "page2", "page3")
print("Input: ", test_sequence)
print("Output: ", predict(test_sequence))
        

Epoch: 1/1000 | Loss: 5.0214
Epoch: 101/1000 | Loss: 2.0295
Epoch: 201/1000 | Loss: 1.4853
Epoch: 301/1000 | Loss: 1.2209
Epoch: 401/1000 | Loss: 1.0716
Epoch: 501/1000 | Loss: 0.9767
Epoch: 601/1000 | Loss: 0.9118
Epoch: 701/1000 | Loss: 0.8652
Epoch: 801/1000 | Loss: 0.8305
Epoch: 901/1000 | Loss: 0.8041
Input:  ('page1', 'page2', 'page3')
Output:  page4


In [41]:
torch.argmax(torch.Tensor([[0., 1., 0., 0., 0.]]), dim=1)

tensor([1])

In [51]:
#dim=1
torch.argmax(tensor 1): tensor([1])
torch.argmax(tensor 1).shape: torch.Size([1])

# dim=0
torch.argmax(tensor 1): tensor([0, 0, 0, 0, 0])
torch.argmax(tensor 1).shape: torch.Size([5])

torch.argmax(tensor 1): 1
torch.argmax(tensor 1).shape: torch.Size([])

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1301496431.py, line 1)